# LangChain Expression Language (LCEL)

A Linguagem de Expressão LangChain, ou LCEL, é uma forma declarativa de **compor cadeias de maneira fácil**. A LCEL foi projetada desde o primeiro dia para suportar a colocação de protótipos em produção, sem a necessidade de alterações no código, desde a cadeia mais simples “prompt + LLM” até as cadeias mais complexas (já vimos pessoas executando com sucesso cadeias LCEL com centenas de etapas em produção). Para destacar alguns dos motivos pelos quais você pode querer usar a LCEL:
- **Suporte a streaming de primeira classe**: menor tempo possível para saída do primeiro token produzidio;
- **Suporte assíncrono**: Qualquer cadeia construída com a LCEL pode ser chamada tanto com a API síncrona;
- **Execução paralela otimizada**: Sempre que suas cadeias LCEL tiverem etapas que podem ser executadas em paralelo, automaticamente é feito isso;
- **Retentativas e fallbacks**: É maneira de tornar suas cadeias mais confiáveis em grande escala, na qual ações alternativas podem ser tomadas no caso de um erro em uma cadeia
- **Acessar resultados intermediários**: auxiliando na depuração de uma cadeia;

## Um exemplo simples de LCEL

O exemplo mais simples que podemos mostrar seria na construção de *prompt + model*

In [ ]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())


In [38]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o')

In [39]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o seguinte: {assunto}')

In [42]:
chain = prompt | model

In [36]:
chain.invoke({'assunto': 'gatinhos'})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Adicinando mais elementos a chain

In [29]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [22]:
chain.invoke({'assunto': 'gatinhos'})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### ATENÇÃO! A ordem importa

In [10]:
chain = model | prompt | output_parser
chain.invoke({'assunto': 'gatinhos'})

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

### Como eu sei a ordem?

Para atingir o mesmo objetivo sem a criação da chain, os passos que eu deveria seguir seriam:
- Formatar o prompt template
- Enviar o prompt formatado para o modelo
- Fazer o parseamento do saída do modelo
Essa mesma ordem deve ser seguida para não termos erros

In [11]:
input = {'assunto': 'gatinhos'}

In [12]:
prompt_formatado = prompt.invoke(input)
prompt_formatado

ChatPromptValue(messages=[HumanMessage(content='Crie uma frase sobre o seguinte: gatinhos', additional_kwargs={}, response_metadata={})])

In [13]:
resposta = model.invoke(prompt_formatado)
resposta

AIMessage(content='"Os gatinhos são a prova de que beleza, fofura e travessura podem coexistir na mesma pelagem."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 19, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2f26491f-341d-4f89-9f68-47b7ba4836fa-0', usage_metadata={'input_tokens': 19, 'output_tokens': 30, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
output_parser.invoke(resposta)

'"Os gatinhos são a prova de que beleza, fofura e travessura podem coexistir na mesma pelagem."'

### Para não errar a ordem

É importante entendermos que cada componente recebe um tipo de entrada e gera um tipo de saída, e estes tipos precisam casar:

| Component       | Tipo de Entrada                                    | Tipo de Saída        |
|-----------------|-----------------------------------------------------|----------------------|
| Prompt          | Dicionário                                         | PromptValue      |
| ChatModel       | String única, lista de mensagens de chat ou PromptValue | Mensagem de Chat     |
| LLM             | String única, lista de mensagens de chat ou PromptValue | String               |
| OutputParser    | A saída de um LLM ou ChatModel                     | Depende do parser    |
| Retriever       | String única                                       | Lista de Documentos  |
| Tool            | String única ou dicionário, dependendo da ferramenta| Depende da ferramenta|

Podemos verificar isso pelos output e input schemas:

In [15]:
print(prompt.input_schema.schema_json(indent=4))

{
    "properties": {
        "assunto": {
            "title": "Assunto",
            "type": "string"
        }
    },
    "required": [
        "assunto"
    ],
    "title": "PromptInput",
    "type": "object"
}


/tmp/ipykernel_141137/370951480.py:1: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(prompt.input_schema.schema_json(indent=4))


In [16]:
chain = prompt | model | output_parser
print(chain.input_schema.schema_json(indent=4))

{
    "properties": {
        "assunto": {
            "title": "Assunto",
            "type": "string"
        }
    },
    "required": [
        "assunto"
    ],
    "title": "PromptInput",
    "type": "object"
}


/tmp/ipykernel_141137/3438426135.py:2: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(chain.input_schema.schema_json(indent=4))


> Atenção!
O método schema_json será removido nas próximas versões do LangChain! É recomendado a utilização do método model_json_schema ao utilizar versões mais recents, da seguinte forma:

In [17]:
chain = prompt | model | output_parser
print(chain.input_schema.model_json_schema())

{'properties': {'assunto': {'title': 'Assunto', 'type': 'string'}}, 'required': ['assunto'], 'title': 'PromptInput', 'type': 'object'}


### Como eu teria feito com chains clássicas?

In [18]:
from langchain.chains.llm import LLMChain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto: {assunto}")
output_parser = StrOutputParser()

In [19]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=output_parser
)
chain.invoke({'assunto': 'gatinhos'})

/tmp/ipykernel_141137/1936627178.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


{'assunto': 'gatinhos',
 'text': '"Os gatinhos são pequenos seres cheios de carinho e travessuras, capazes de alegrar qualquer coração com seu jeito leve e brincalhão."'}